In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
from ovito.io import import_file, export_file
from ovito.modifiers import CoordinationAnalysisModifier, TimeAveragingModifier, SelectTypeModifier

In [2]:
Temps=(50,100,200,400,600,800)
colors = mpl.colormaps.get_cmap('viridis').resampled(6).colors
nbins=200
rcut=4

### Use Ovito for doing time-averaged rdf calculation

In [4]:
for i, T in enumerate(Temps):
    color=colors[i]
    file="T_scan/"+str(T)+"K/dump.xyz"
    # Load a simulation trajectory consisting of several frames:
    pipeline = import_file(file)
    print("Number of MD frames:", pipeline.source.num_frames)

    # Select by type
    pipeline.modifiers.append(SelectTypeModifier(
        operate_on = "particles",
        property = "Particle Type",
        types = {2, 3} #Ni, O
    ))      

    # Insert the RDF calculation modifier into the pipeline:
    pipeline.modifiers.append(CoordinationAnalysisModifier(cutoff = rcut, number_of_bins = nbins,only_selected=True))
    # Insert the time-averaging modifier into the pipeline, which accumulates
    # the instantaneous DataTable produced by the previous modifier and computes a mean histogram.
    pipeline.modifiers.append(TimeAveragingModifier(operate_on='table:coordination-rdf'))
    # Data export method 1: Convert to NumPy array and write data to a text file:
    rdf_ovito = pipeline.compute().tables['coordination-rdf[average]'].xy()
    file_out="T_scan/"+str(T)+"K/rdf_Ni_O.txt"
    np.savetxt(file_out,rdf_ovito)
    #r=np.linspace(0,rcut,nbins)
    #rdf=np.loadtxt(file_out,usecols=(1,),unpack=True)
    #plt.plot(r,rdf,marker='o',linestyle='none',color=color)
#plt.xlabel("r $[\AA]$",fontsize=14)
#plt.ylabel("g(r)",fontsize=14)
#plt.xlim(1,4)
#plt.legend()
#plt.show()

Number of MD frames: 10004


FileNotFoundError: [Errno 2] No such file or directory: 'T_scan/50/rdf_Ni_O.txt'